#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, KFold
import time
from scipy.stats import spearmanr

#### Import Data

In [2]:
ratings = pd.read_csv('../../ratings.csv')

#### Make Subsets of Data for Part I

In [58]:
def pick_users_books(df, num_users, num_books):
    user_counts = pd.DataFrame(df.user_id.value_counts()).sort_values('user_id', ascending=False)
    top_10K_users = list(user_counts[0:num_users].index)
    user_filtered_df = df[df.user_id.isin(top_10K_users)]
    filtered_book_counts = pd.DataFrame(user_filtered_df.book_id.value_counts()).sort_values('book_id', 
                                                                                             ascending = False)
    top_100_filtered_books = list(filtered_book_counts[0:num_books].index)
    filtered_df = user_filtered_df[user_filtered_df.book_id.isin(top_100_filtered_books)]
    train, test = train_test_split(filtered_df, test_size = 0.2, random_state=42)
    return train, test
    
def get_all_subsets(df):
    train_500_20, test_500_20 = pick_users_books(df, 500, 20)
    train_1000_35, test_1000_35 = pick_users_books(df, 1000, 35)
    train_2000_50, test_2000_50 = pick_users_books(df, 2000, 50)
    train_5000_70, test_5000_70 = pick_users_books(df, 5000, 70)
    train_7500_85, test_7500_85 = pick_users_books(df, 7500, 85)
    train_10000_100, test_10000_100 = pick_users_books(df, 10000, 100)
    return train_500_20, test_500_20, train_1000_35, test_1000_35, train_2000_50, test_2000_50, \
        train_5000_70, test_5000_70, train_7500_85, test_7500_85, train_10000_100, test_10000_100

In [60]:
train_500_20, test_500_20, train_1000_35, test_1000_35, train_2000_50, test_2000_50, train_5000_70, test_5000_70, \
    train_7500_85, test_7500_85, train_10000_100, test_10000_100 = get_all_subsets(ratings)

#### Implement Matrix Factorization

In [5]:
def preprocess(X_train, X_test):
    
    # create user and book indices starting from 0
    mappings_user = pd.DataFrame({'user_id': sorted(X_train.user_id.unique()), 
                              'user_idx': range(len(X_train.user_id.unique()))})
    mappings_book = pd.DataFrame({'book_id': sorted(X_train.book_id.unique()), 
                              'book_idx': range(len(X_train.book_id.unique()))})
    X_train = pd.merge(X_train, mappings_user, on='user_id')
    X_train = pd.merge(X_train, mappings_book, on='book_id')
    X_test = pd.merge(X_test, mappings_user, on='user_id')
    X_test = pd.merge(X_test, mappings_book, on='book_id')
    
    # create user-item matrix for training data and find non-zero values
    M = np.array(X_train.pivot_table(index = 'user_idx', columns='book_idx', values='rating', fill_value=0))
    mask = M != 0
    
    # use fake data to make test matrix that matches the size of the train matrix
    fake_book = pd.DataFrame({'user_id': sorted(X_train.user_id.unique()), 'rating': 0,
                          'user_idx': range(len(X_train.user_id.unique())), 'book_id': 'XXX', 'book_idx': 100})
    fake_user = pd.DataFrame({'book_id': sorted(X_train.book_id.unique()), 'rating': 0,
                          'book_idx': range(len(X_train.book_id.unique())), 'user_id': 'XXX', 'user_idx': 10000000000})
    X_test = pd.concat([X_test, fake_book, fake_user])
    M_test = X_test.pivot_table(index = 'user_idx', columns='book_idx', values='rating', fill_value=0)
    M_test.drop(100, axis=1, inplace=True)
    M_test.drop(10000000000, axis=0, inplace=True)
    
    # subtract off user means
    means_list = []
    for row in range(M.shape[0]):
        n_ratings = len(np.where(M[row,:] != 0)[0])
        means_list.append(np.sum(M[row,:]) / n_ratings)
    means = np.array(means_list).reshape(-1,1)
    M_norm = (M - means) * mask
    
    return (M_norm, means, np.array(M_test))

In [6]:
def update(R, mask, U, V, alpha, E, lamb): # performs one iteration of updating U and V matrices
    U_new = (U * (1 - (alpha * lamb))) + (alpha * np.dot(E, V))
    V_new = (V * (1 - (alpha * lamb))) + (alpha * np.dot(E.transpose(), U))
    return (U_new, V_new)

In [7]:
def calc_error(R, mask, U, V): 
    # calculates error matrix - difference between prediction and true rating where true rating exists, 0 otherwise
    E = (R - np.dot(U, V.transpose())) * mask
    return E

In [8]:
def calc_loss(E, U, V, lamb): # calculate value of loss function based on error matrix
    J = (0.5 * np.sum(np.square(E))) + ((lamb/2) * np.sum(np.square(U))) + ((lamb/2) * np.sum(np.square(V)))
    return J

In [20]:
def mat_fact(R, d, alpha, lamb):
    # initialize U and V
    U = np.random.randn(R.shape[0], d)
    V = np.random.randn(R.shape[1], d)
    
    # calculate error and loss
    mask = R != 0
    E = calc_error(R, mask, U, V)
    J_prev = calc_loss(E, U, V, lamb)
    J_ratio = 1
    
    # while not converged, update U and V and recalculate error and loss
    count = 0
    while np.abs(J_ratio) > .00001:
        if count > 1000 and J_prev > 10000:
            print('Did not converge!')
            return (U, V)
        else:
            U, V = update(R, mask, U, V, alpha, E, lamb)
            E = calc_error(R, mask, U, V)
            J = calc_loss(E, U, V, lamb)
            J_ratio = (J_prev - J) / J_prev
            J_prev = J
            count += 1
            
    return (U, V)

In [10]:
def calc_RMSE(M_test, preds): # calculate RMSE using only observed entries
    mask_test = M_test != 0
    preds_masked = mask_test * preds
    rmse = np.sqrt(np.sum(np.sum(np.square(preds_masked - M_test))) / np.sum(np.sum(mask_test)))
    return rmse

#### Grid-Search Parameters
Use only biggest subset

In [11]:
# d_values = [5, 10, 15]
# alpha_values = [0.00005, 0.0001, 0.00015, 0.0002]
# lamb_values = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]

In [12]:
# def overall(data, d, alpha, lamb):
#     k_fold = KFold(n_splits=3)
#     rmses = []
#     for train_indices, test_indices in k_fold.split(data):
#         X_train = data.iloc[train_indices]
#         X_test = data.iloc[test_indices]
#         M_norm, means, X_test = preprocess(X_train, X_test)
#         U, V = mat_fact(M_norm, d, alpha, lamb)
#         preds = np.dot(U, V.transpose()) + means
#         rmse = calc_RMSE(X_test, preds)
#         rmses.append(rmse)
#     return np.mean(rmses)

In [22]:
# grid_results_all = []
# for d in d_values:
#     print('Current d value: ', d)
#     for alpha in alpha_values:
#         print('Current alpha value: ', alpha)
#         for lamb in lamb_values:
#             print('Current lambda value: ', lamb)
#             grid_results = {}
#             rmse = overall(train_10000_100, d, alpha, lamb)
#             grid_results['d'] = d
#             grid_results['alpha'] = alpha
#             grid_results['lambda'] = lamb
#             grid_results['rmse'] = rmse
#             grid_results_all.append(grid_results)

Current d value:  5
Current alpha value:  5e-05
Current lambda value:  1e-05
Current lambda value:  0.0001
Current lambda value:  0.001
Current lambda value:  0.01
Current lambda value:  0.1
Current lambda value:  1
Did not converge!
Did not converge!
Did not converge!
Current alpha value:  0.0001
Current lambda value:  1e-05
Current lambda value:  0.0001
Current lambda value:  0.001
Current lambda value:  0.01
Current lambda value:  0.1
Current lambda value:  1
Did not converge!
Did not converge!
Did not converge!
Current alpha value:  0.00015
Current lambda value:  1e-05
Current lambda value:  0.0001
Current lambda value:  0.001
Current lambda value:  0.01
Current lambda value:  0.1
Current lambda value:  1
Did not converge!
Did not converge!
Did not converge!
Current alpha value:  0.0002
Current lambda value:  1e-05
Current lambda value:  0.0001
Current lambda value:  0.001
Current lambda value:  0.01
Current lambda value:  0.1
Current lambda value:  1
Did not converge!
Did not conv

In [61]:
# grid_df = pd.DataFrame(grid_results_all)
# grid_df
# grid_df.to_pickle('grid_results.p')

In [55]:
# grid_df.iloc[grid_df['rmse'].idxmin()]

alpha     0.000100
d         5.000000
lambda    0.001000
rmse      0.958122
Name: 8, dtype: float64

In [56]:
# min_values = grid_df.iloc[grid_df['rmse'].idxmin()]
# print('The best parameters were d = {}, alpha = {}, and lambda = {}, with an average RMSE of {}.'
#       .format(min_values['d'], min_values['alpha'], min_values['lambda'], min_values['rmse']))

# # RESULT: The best parameters were d = 5.0, alpha = 0.0001, and lambda = 0.001, with an average RMSE of 0.9581215576515847.

The best parameters were d = 5.0, alpha = 0.0001, and lambda = 0.001, with an average RMSE of 0.9581215576515847.


#### Train 6 Subsets to Observe Scalability

In [62]:
def calc_MAE(M_test, preds): # calculate MAE using only observed entries
    mask_test = M_test != 0
    preds_masked = mask_test * preds
    mae = np.sum(np.sum(np.abs(preds_masked - M_test))) / np.sum(np.sum(mask_test))
    return mae

In [63]:
def calc_spearman(M_test, preds): # calculate spearman coefficient using only observed entries
    spearmans = []
    for i in range(len(M_test)):
        mask = M_test[i,:] != 0
        if sum(mask) > 1: # can't calculate if there's only one item rated
            M_test_mask = M_test[i, mask]
            preds_mask = preds[i, mask]
            spearman = spearmanr(M_test_mask, preds_mask)[0]
            if np.isnan(spearman) == False: # spearman is NaN if all true ratings are the same! exclude these
                spearmans.append(spearman)
    return np.mean(spearmans)

In [67]:
def train_test(X_train, X_test):
    M_norm, means, X_test = preprocess(X_train, X_test)
    start_train = time.time()
    U, V = mat_fact(M_norm, 5, 0.0001, 0.001)
    end_train = time.time()
    start_pred = time.time()
    preds = np.dot(U, V.transpose()) + means
    end_pred = time.time()
    rmse = calc_RMSE(X_test, preds)
    mae = calc_MAE(X_test, preds)
    spearman = calc_spearman(X_test, preds)
    train_time = end_train - start_train
    pred_time = end_pred - start_pred
    return (train_time, pred_time, rmse, mae, spearman)

In [65]:
subsets = [(train_500_20, test_500_20, 500, 20), (train_1000_35, test_1000_35, 1000, 35), 
           (train_2000_50, test_2000_50, 2000, 50), (train_5000_70, test_5000_70, 5000, 70),
           (train_7500_85, test_7500_85, 7500, 85), (train_10000_100, test_10000_100, 10000, 100)]

In [70]:
scalability_all = []
for (X_train, X_test, n_users, n_items) in subsets:
    scalability = {}
    train_time, pred_time, rmse, mae, spearman = train_test(X_train, X_test)
    scalability['Number of Users'] = n_users
    scalability['Number of Items'] = n_items
    scalability['Time to Train'] = train_time
    scalability['Time to Predict'] = pred_time
    scalability['RMSE'] = rmse
    scalability['MAE'] = mae
    scalability['Spearman'] = spearman
    scalability_all.append(scalability)

In [71]:
pd.DataFrame(scalability_all)

,MAE,Number of Items,Number of Users,RMSE,Spearman,Time to Predict,Time to Train
0,0.722292,20,500,0.934416,0.002052,0.000043,0.087984
1,0.744592,35,1000,0.969022,-0.022090,0.000178,0.128904
2,0.739621,50,2000,0.956829,0.024255,0.000336,0.207256
3,0.722454,70,5000,0.931651,0.006256,0.002735,0.466233
4,0.736573,85,7500,0.943755,0.000212,0.004573,0.674893
5,0.739497,100,10000,0.949419,0.002610,0.008565,1.053142
